In [ ]:
# Import Libraries
import sqlite3
import pandas as pd
import numpy as np
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
# Load txt files into pandas and separate columns 
adhesiveSealantTrans = pd.read_csv("/path/to/file/containing/all/transactions", header=0)
allTransaction = pd.read_csv("/path/to/file/containing/all/transactionst", header=0)

In [ ]:
# Verify the data loaded correctly
adhesiveSealantTrans.head()

In [ ]:
# Verify the data loaded correctly
allTransaction.head()

In [ ]:
# Combine transaction data since adhesives/sealants do not need to be seperate from all other transactions
frames = [adhesiveSealantTrans, allTransaction]
allTransactions = pd.concat(frames)

# Check data loaded correctly
allTransactions.head()

In [ ]:
# Drop sku and brand 
allTransactions.drop('sku', axis=1, inplace=True)
allTransactions.drop('brand', axis=1, inplace=True)

# Drop most broad categories and keep most granular
allTransactions.drop('l1', axis=1, inplace=True)
allTransactions.drop('l2', axis=1, inplace=True)

In [ ]:
grouped = allTransactions.groupby('order_number')['l3'].apply(list)
grouped.head()

In [ ]:
#Filter out small orders
filtered_group = [x for x in grouped if len(x)>=10 ]

# Sample first few large orders
filtered_group[:3]

In [ ]:
# One-hot encode data in pandas dataframe
te = TransactionEncoder()
te_ary = te.fit(filtered_group).transform(filtered_group)
transaction_group = pd.DataFrame(te_ary, columns=te.columns_)
transaction_group.head()

In [ ]:
# Find items with minimal support using apriori algorithm to show which have any possible relationship
apriori(transaction_group, min_support=0.01, use_colnames=True)

In [ ]:
# Find frequent item sets and and number of items in each order
freq_itemsets = apriori(transaction_group, min_support=0.01, use_colnames=True)
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x : len(x))
freq_itemsets.head()

In [ ]:
#Filter out single-item orders 
freq_itemsets[ (freq_itemsets['length'] > 1) &
             (freq_itemsets['support'] > 0.02) ]

In [ ]:
# Generate association rules to make recommendations using lift metric to choose items that are purchased together 
# more often than expected
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)
rules.head()

In [ ]:
# Filter items and save
saved_recs = rules[ (rules['lift'] >= 6) &
     rules['confidence'] >= 0.8]

# Check saved dataframe
saved_recs.sort_values(by=['confidence'], ascending=False)

In [ ]:
# Save Recs to csv
saved_recs.to_csv('recommendations.csv', index=False)